# Spotify Tracks Dataset
__Aprendizaje Automático Avanzado (AAA)__

_Alan García Justel_

Paara el desarrollo de esta práctica, se ha elegido el dataset [Spotify Tracks Dataset](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset), el cual contiene información de al rededor de $125$ géneros de música distintos. Este dataset puede ser utilizado para desarrollar aplicaciones de recomentación, clasificación de canciones o ingluso de predicción de popularidad.

En este notebook se va a explorar el cómo emplear las features descriptivas para realizar clasificaciones en base al género de las canciones. Estas features son tabulares y han sido recogidas de diferentes formas que se detallarán más adelante. No se va a hacer uso de las propias pistas de audio, ya que para esa labor han de intervenir técnicas de procesamiento de series temporales que se escapan del alcanze de la asignatura. Es por ello, que el objetivo recogido en el siguiente cuaderno es el de conseguir un sistema de clasificación basado en características de las canciones previamente anotadas.

## Setup

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import warnings
import os

# warnings.filterwarnings("ignore", category=UserWarning)

## Dataset Exploration

El dataset se proporciona en un fichero _.csv_, por lo que se ha decidido utilizar la librería de `pandas` para manejar los datos y realizar una exploración inicial de las variables presentadas. Concretamente, el dataset cuenta con $114000$ instancias y $21$ features:
    
* __Unamed0__: Un identificador dado a cada una de las instancias por los creadores del dataset.
* __track_id__: El ID del track para acceder a la secuencia de audio de la canción. 
* __artists__: El o los nombres de los artistas de la canción. Si son varios están separados por _;_.
* __album_name__: El álbum de la canción.
* __track_name__: Nombre de la canción.
* __popularity__: The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.
* __duration_ms__: Duración de la canción en ms.
* __explicit__: Whether or not the track has explicit lyrics (true = yes it does; false = no it does not OR unknown)
* __danceability__: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable
* __energy__: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale
* __key__: The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1
* __loudness__: The overall loudness of a track in decibels (dB)
* __mode__: Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0
* __speechiness__: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks
* __acousticness__: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic
* __instrumentalness__: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content
* __liveness__: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live
* __valence__: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)
* __tempo__: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration
* __time_signature__: An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4.
* __track_genre__: The genre in which the track belongs

In [ ]:
df = pd.read_csv('../../data/AAA/spotify_tracks.csv')
print(f"data_frame shape: {df.shape}")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

Al encontrarnos con estos datos nos pueden surgir varias cuestiones:
- ¿Cuántos géneros musicales distintos presenta el dataset? 
- ¿Las clases predictoras están balanceadas?
- ¿Hay missing values? De ser así, ¿cómo los tratamos?
- ¿Hay entradas duplicadas? De ser así, ¿qué hacemos con ellas?
- ¿Cómo de correlacionadas están las distintas clases predictoras?
- ¿Qué relación existe entre las variables descriptivas como el _tempo_, por ejemplo, y los géneros musicales de las canciones?
- ¿Realizaremos un estudio y modelos predictores para todas las clases o solo nos centraremos en un subconjunto de ellas?

In [ ]:
counts = df['track_genre'].value_counts()
print(f"El dataset presenta {len(counts)} géneros musicales distintos.")

distinc_nums = []
for c in counts:
    if c not in distinc_nums:
        distinc_nums.append(c)

if len(distinc_nums) == 1:
    print(f"El dataset está balanceado y cada género musical tiene {distinc_nums[0]} instancias.")
else:
    print(f"El dataset not está balanceado. Hay géneros con estos números de instancias: {distinc_nums}.")

Existe una instancia del dataset que tiene todos los datos exceptuando aquellos relacionados con los nombres del artista, el álbum y el nombre de la canción. Sin embargo, sí que contamos con el _track_id_ de la canción, por lo que podemos hacer una consulta rápida mediante la _API_ de spotify para recuperar los datos faltantes de la canción.

In [ ]:
exist_missing_values = False
for i, is_null in enumerate(df.isnull().sum()):
    if is_null:
        exist_missing_values = True
        print(f"La variable '{df.columns[i]}' presenta {is_null} missing values")

df[df.isnull().any(axis=1)]

A pesar de ello, al hacer la consulta a la _API_ de Spotify, vemos que los datos de la canción siguen estando omitidos. Por ello, considerando que solo se trata de un único caso, de momento se va a asumir que los demás datos son correctos ya que el resto de datos parecen correctos a simple vista. Quizá más adelante resulte que esta instancia sea un outlier o ruido que deba ser tratado.

In [ ]:
def buscar_datos_cancion(track_id:str):
    """
    Se ha de contar con las variables de entorno SPOTIFY_CLIENT_ID y 
    SPOTIFY_CLIENT_SECRET de Spotify Developers para poder acceder 
    a la API de Spotify.
    """
    import spotipy
    from spotipy.oauth2 import SpotifyClientCredentials

    # Configura tus credenciales
    CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
    CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
    if not CLIENT_ID or not CLIENT_SECRET:
        raise ValueError("Asegúrate de que las variables de entorno SPOTIFY_CLIENT_ID y SPOTIFY_CLIENT_SECRET están configuradas.")

    # Autenticación
    auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(auth_manager=auth_manager)

    # Imprimir el nombre de la canción y el artista
    track_info = sp.track(track_id)
    print(f"Nombre de la canción: {track_info['name']}")
    print(f"Artista: {'; '.join(artist['name'] for artist in track_info['artists'])}")

buscar_datos_cancion(track_id="1kR4gIb7nGxHPI3D2ifs59")

Por otro lado, no existen instancias duplicadas :)

In [ ]:
duplicates = df.duplicated().sum()
print(f"Existen {duplicates} instancias duplicadas.")

Con todo esto, vamos a centrarnos en aquellas variables relevantes para desarrollar la aplicación de clasificación de canciones. Por ello, las variables `"Unnamed: 0"`, `"track_id"`, `"artists"`, `"album_name"` y `"track_name"` no se van a tener en cuenta. Además, también se va a codificar la variable predictora asignando un identificador numérico a cada género musical.

In [ ]:
# Codificar la variable 'track_genre' para análisis
LE = LabelEncoder()
df["track_genre_encoded"] = LE.fit_transform(df["track_genre"])

columns_to_drop = ["Unnamed: 0", "track_id", "artists", "album_name", "track_name", "track_genre"]
for c in columns_to_drop:
    if c in df.columns:
        df = df.drop(columns=c)

# Convertir todas las variables a float por consistencia de tipos
for col in df.select_dtypes(include=["int64"]).columns:
    df[col] = df[col].astype(float)

In [ ]:
correlation_matrix = df.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Mapa de calor de correlaciones (incluyendo género codificado)")
plt.show()

print("Valores de las correlaciones entre las variables y el género musical")
print(correlation_matrix["track_genre_encoded"].sort_values(ascending=False))

Lo primero que salta a la vista al observar esta matriz de correlaciones es que no está muy clara la relación entre las variables y el género musical. La intuición inicial planteaba la siguiente hypótesis inductiva: "Las canciones de rock suelen tener mucha energía y suelen tener un volumen alto, mientras que las pistas de música clásica son más relajadas, por lo que existe una correlación fuerte entre las variables tabulares y los géneros musicales". Sin embargo, podemos sacar algunas conclusiones preliminares a partir de esta matriz de correlaciones:
- La durabilidad de las canciones está devilmente correlacionada con 

In [ ]:
# Estandarizar los datos
non_feature_columns = ["track_genre_encoded"]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.drop(columns=non_feature_columns))
df_scaled = pd.DataFrame(scaled_features, columns=df.drop(columns=non_feature_columns).columns)
df_scaled['track_genre_encoded'] = df['track_genre_encoded'].values
df = df_scaled

# Agrupar por género
genre_profiles = df.groupby('track_genre_encoded').mean()

# Calcular una matriz de distancia entre géneros
# metric: ['euclidean', 'cosine', 'correlation']
distance_matrix = pdist(genre_profiles, metric='euclidean')
distance_df = pd.DataFrame(
    squareform(distance_matrix),
    index=genre_profiles.index,
    columns=genre_profiles.index
)

# Visualizar la matriz de distancias
plt.figure(figsize=(12, 10))
sns.heatmap(distance_df, cmap='coolwarm', xticklabels=True, yticklabels=True)
plt.title('Similitud entre géneros basada en características')
plt.show()

In [ ]:
# Crear una lista de pares de géneros con sus distancias
processed_pairs = set()
pairs = []
for genre1 in distance_df.columns:
    for genre2 in distance_df.columns:
        if genre1 != genre2:
            # Esto se puede hacer porque estamos considerando una funcion de distancia es simétrica            
            if (genre1, genre2) not in processed_pairs and (genre2, genre1) not in processed_pairs:
                pairs.append((int(genre1), int(genre2), distance_df.loc[genre1, genre2]))
                processed_pairs.add((genre1, genre2))
                

# Convertir la lista en un DataFrame
pairs_df = pd.DataFrame(pairs, columns=['genre_encoded_1', 'genre_encoded_2', 'distance'])
pairs_df['genre_1'] = LE.inverse_transform( pairs_df['genre_encoded_1'] )
pairs_df['genre_2'] = LE.inverse_transform( pairs_df['genre_encoded_2'] )
sorted_pairs = pairs_df.sort_values(by='distance')

print("Lista de pares de géneros musicales más similares en base a la disimilaridad calculada:")
pd.set_option('display.max_rows', None)  # Muestra todas las filas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.width', None)  # Evita el corte de texto largo
print(sorted_pairs)

Estamos calculando la matriz de similitudes en base a las distancias de las medias normalizadas de cada una de las variables agrupadas por género. Es por ello que surge la siguiente cuestión: ¿Son la media y la desviación típica estadísticos representativos de las variables?

Para que fuese así, los datos deberían de estar simétricamente distribuidos, no deberían de estar sesgados (poca presencia de outliers) y no deberían de ser asimétricos. Por ello, se ha decidido representar las distribuciones de cada una de las variables en un violin plot en el cual se puede distinguir visualmente. Así, se puede observar cómo muy pocas variables se aproximan a una distribución normal a excepción  de las variables de duración y "bailabilidad", las cuales sí se asemejan en cierta manera a una distribución normal (al menos de forma visual). Además, 


La variable `mode` es booleana, no es representativo el violin plot. 
La variable `time_signature` toma valores discretos 0-7, no es representativo el violin plot.
La variable `liveness` o de presencia de multitudes en el audio parece presentar outliers.
La variable `speechiness` también parece presentar outliers a la derecha.


En este punto nos encontramos con dos vías de estudio principales. En primer lugar, hay que comprobar si efectivamente las variables de duración y "bailabilidad" siguen distribuciones normales o es solo.

In [ ]:
from scipy.stats import skew, kurtosis, shapiro

selected_columns = ['acousticness', 'valence', 'liveness', 'popularity', 'danceability', 'key', 
                    'tempo', 'loudness', 'duration_ms', 'energy', 'instrumentalness', 'speechiness' ]
# Non selected columns: track_genre_encoded, mode, explicit, time_signature
print(len(selected_columns))
fig, axes = plt.subplots(4, 3, figsize=(20, 16))
axes = axes.flatten()


# Violin plot para cada variable numérica
stat_data = {}
for i, column in enumerate(selected_columns):
    sns.violinplot(x=df[column], color='lightblue', ax=axes[i])
    axes[i].set_title(f'Distribución de {column}')
    axes[i].set_xlabel(column)

    # Cálculo de asimetría y curtosis
    mean = df[column].mean()
    std_dev = df[column].std()
    column_skewness = skew(df[column].dropna())
    column_kurtosis = kurtosis(df[column].dropna())
    stat, p_value = shapiro(df[column].dropna())
    
    stat_data[column] = {
        "mean": mean,
        "std_dev": std_dev,
        "skewness": column_skewness,
        "kurtosis": column_kurtosis,
        "shapiro":{"stat": stat, "p_value": p_value}
    }

plt.tight_layout()
plt.show()

In [ ]:
for sd in stat_data.keys():
    if stat_data[sd]['shapiro']['p_value'] > 1e-10:
        print(stat_data[sd])

In [ ]:
stat_data['duration_ms']

Ninguna variable parece presentar un p-valor superior a $1 \times 10^{-10}$ en el test de _Saphiro-Wilk_, por lo que es estadisticamente significativo el decir que ninguna de ellas se aproxima a una distribución normal. Por otro lado, es interesante realizar este mismo estudio visual agrupando los datos por su género musical correspondiente ya que puede darse el caso de que las anomalías y distintas distribuciones se deban al agrupamiento y el estudio global de todos los datos sin tener en cuenta los géneros musicales de cada instancia.
Sin embargo, al haber 114 géneros musicales distintos, es complicada la visualización en este tipo de diagrama. 


Por ello, se proponen dos métodos para realizar la visualización:
1. Agrupar los géneros musicales empleando un algoritmo de clustering aglomerativo generando clusters en base a la matriz de similaridad expuesta anteriormente.
2. Seleccionar de forma aleatoria un género musical y visualizar su distribución

In [ ]:
n_clusters = 15 # partiendo de 114
clustering = AgglomerativeClustering(n_clusters=n_clusters, metric='precomputed', linkage='complete')
cluster_labels = clustering.fit_predict(squareform(distance_matrix))

# Crear un DataFrame para ver cómo se agrupan los géneros
clustered_genres = pd.DataFrame({
    'track_genre_encoded': distance_df.columns,
    'cluster': cluster_labels
})

# Asignar los nuevos clusters a los géneros musicales
clustered_genres['track_genre'] = LE.inverse_transform(clustered_genres['track_genre_encoded'].astype(int))
print("Agrupaciones de géneros musicales calculadas:")
CE = clustered_genres.groupby('cluster')['track_genre'].apply(list) # Cluster Encoder
print(CE)

metal_assert = ['black-metal', 'death-metal', 'grindcore', 'heavy-metal', 'industrial', 'metalcore']
assert CE[0] == metal_assert

# Hacer un violin plot por cada variable y cada cluster agrupando los datos
df['cluster'] = df['track_genre_encoded'].map(clustered_genres.set_index('track_genre_encoded')['cluster'])
selected_columns = ['acousticness', 'valence', 'liveness', 'popularity', 'danceability', 'key', 
                    'tempo', 'loudness', 'duration_ms', 'energy', 'instrumentalness', 'speechiness']

fig, axes = plt.subplots(4, 3, figsize=(20, 16))
axes = axes.flatten()
for i, column in enumerate(selected_columns):
    sns.violinplot(x='cluster', y=column, data=df, color='purple', inner="quart", ax=axes[i])
    axes[i].set_title(f'Distribución de {column} por cluster')
    axes[i].set_xlabel(column)
plt.tight_layout()
plt.show()


In [ ]:
selected_columns = ['acousticness', 'valence', 'liveness', 'popularity', 'danceability', 'key', 
                    'tempo', 'loudness', 'duration_ms', 'energy', 'instrumentalness', 'speechiness' ]
# Non selected columns: track_genre_encoded, mode, explicit, time_signature
print(len(selected_columns))
selected_genre = np.random.randint(low=0, high=len(LE.classes_))
filtered_df = df[df['track_genre_encoded'] == selected_genre]

# Crear un gráfico de violín para cada columna seleccionada
fig, axes = plt.subplots(4, 3, figsize=(20, 16))
axes = axes.flatten()
for i, column in enumerate(selected_columns):
    plt.figure(figsize=(10, 6))  # Tamaño de cada gráfico
    sns.violinplot(y=column, data=filtered_df, inner="quart", color="lightgreen", ax=axes[i])
    axes[i].set_title(f'Distribución de {column} para el la clase {LE.classes_[selected_genre]}')
    axes[i].set_xlabel(column)
    
plt.tight_layout()
plt.show()

La conclusión general es que ni agrupando por clusters o por clases se consiguen datos distribuidos de forma normal. Esto puede deberse a que las canciones, aunque pertenezcan a un género en específico, pueden contener rasgos de otros géneros musicales. No todo es blanco o negro. Es por ello que también es interesante detectar estos outliers para ver cómo se comportan. Además, a la hora de hacer un clasificador quizás no sea lo óptimo uno que clasifique canciones proporcionando un único género musical como salida, si no un abanico de distintos géneros musicales con mayor o menor probabilidades.

Considerando este análisis, los clústeres de agrupación obtenido y los objetivos didácticos de esta práctica, se propone estudiar un problema de clasificación binaria en el que se pretende detectar si una instancia pertenece al siguiente grupo de canciones o no:


In [ ]:
clustered_genres.groupby('cluster')['track_genre'].apply(list)[0]

In [ ]:
# Create the binary target variable
df['is_metal'] = df['cluster'].apply(lambda x: 1 if x == 0 else 0)
df.head()

In [ ]:
metal_data = df[df['is_metal'] == 1]

non_feature_columns = ["track_genre_encoded", "cluster", "is_metal"]
metal_numeric = metal_data.drop(columns=non_feature_columns)
correlation_matrix = metal_numeric.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Matriz de correlación de las variables para el clúster "Metal"')
plt.show()

En este clúster se recogen los géneros de la conocida como música _metal_ y que está caracterizada por su alta _energía_...

In [ ]:
non_feature_columns = ["track_genre_encoded", "cluster", "is_metal"]
X = df.drop(columns=non_feature_columns)  # Features
y = df['is_metal']  # Target

## Visualización de los datos

In [ ]:
# Aplicar PCA
pca = PCA()
pca_result = pca.fit_transform(X)
explained_variance = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance)
print("PCA Varianza Acumulada:\n", cumulative_explained_variance)

# Dataframe con los datos del PCA para los plots
pca_df = pd.DataFrame(data=pca_result[:, :3], columns=['PC1', 'PC2', 'PC3'])
pca_df['cluster'] = df['cluster'].values

# Mostrar el scree plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='--')
plt.xlabel('Número de componentes principales')
plt.ylabel('Autovalor')
plt.title('Scree Plot')
plt.show()

# 2D Features con PCA
plt.figure(figsize=(10, 8))
plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['cluster'], cmap='tab20', alpha=0.6)
plt.colorbar(label='Cluster')
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title(f"2D PCA Plot con varianza acumulada de {cumulative_explained_variance[1]}")
plt.show()

El grafico no es nada representativo. Varianza acumulada muy baja y tampoco mejora con las 3 primeras componentes principales. Este método de visualización no es adecuado en este caso y no nos aporta nueva información. Ya sabíamos que las clases parecen estar muy aglomeradas y los géneros musicales normalmente combinan elementos unos de otros.

In [ ]:
tsne = TSNE(n_components=2, perplexity=30, random_state=42, learning_rate=200, max_iter=1000)
tsne_result = tsne.fit_transform(X)

# Create a DataFrame with t-SNE results
tsne_df = pd.DataFrame(data=tsne_result, columns=['TSNE1', 'TSNE2'])
tsne_df['track_genre_encoded'] = df['track_genre_encoded'].values
tsne_df['cluster'] = df['cluster'].values

# t_SNE Clusters
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
scatter1 = axes[0].scatter(tsne_df['TSNE1'], tsne_df['TSNE2'], c=tsne_df['cluster'], cmap='tab20', alpha=0.6)
axes[0].set_title("t-SNE por cada clúster")
axes[0].set_xlabel("t-SNE Dimensión 1")
axes[0].set_ylabel("t-SNE Dimensión 2")
fig.colorbar(scatter1, ax=axes[0], label='Cluster')

# t_SNE Genres
scatter2 = axes[1].scatter(tsne_df['TSNE1'], tsne_df['TSNE2'], c=tsne_df['track_genre_encoded'], cmap='tab20', alpha=0.6)
axes[1].set_title("t-SNE por cada género etiquetado")
axes[1].set_xlabel("t-SNE Dimensión 1")
axes[1].set_ylabel("t-SNE Dimensión 2")
fig.colorbar(scatter2, ax=axes[1], label='Genre')

plt.tight_layout()
plt.show()

## Detección de outliers

Como se ha comentado, la asignación de un género musical a una canción no siempre es clara. Además, en este caso de estudio, estamos considerando el género musical "metal" para realizar el estudio, género que a su vez es una agrupación de otros subgéneros y ya se ha visto como las variables descriptivas de las canciones no siguen una distribución normal, son asimétricas y parecen presentar outliers.
Por esta razón es muy interesante determinar con qué tipo de outliers nos podemos encontrar y determinar si a la hora de entrenar un clasificador es mejor considerar o no considerar estas canciones.

Una primera opción que, aunque sencilla, se ve interesante es el de asumir que las features deben de seguir una distribución normal...

In [ ]:
from scipy.stats import zscore
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Supongamos que df es tu DataFrame con las características normalizadas o no normalizadas

# Paso 1: Calcular Z-Score para todas las features
z_scores = np.abs(zscore(X))

# Paso 2: Identificar outliers
threshold = 3  # Umbral para considerar un outlier
outliers = (z_scores > threshold).any(axis=1)

# Agregar columna indicando si el punto es un outlier

# Paso 3: Resumen de outliers detectados
print(f"Total de outliers detectados: {df['is_outlier'].sum()}")

# Paso 4: Visualización de outliers (usando PCA para reducir a 2D si es necesario)
from sklearn.decomposition import PCA

# Reducir a 2D para visualizar
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X)

# Crear DataFrame con PCA y etiquetas de outliers
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df['is_outlier'] = outliers

# Graficar puntos normales y outliers
plt.figure(figsize=(10, 8))
plt.scatter(pca_df[~pca_df['is_outlier']]['PC1'], pca_df[~pca_df['is_outlier']]['PC2'], alpha=0.6, label='Normal', c='blue')
plt.scatter(pca_df[pca_df['is_outlier']]['PC1'], pca_df[pca_df['is_outlier']]['PC2'], alpha=0.6, label='Outlier', c='red')
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Detección de Outliers usando Z-Score")
plt.legend()
plt.show()


Detección de outliers basado en vecindad. Concretamente se va a calcular el Local Outlier Factor o LOF

In [ ]:
### LocalOutlierFactor #####################################################################
clf = LocalOutlierFactor(n_neighbors=20)
outliers_pred = clf.fit_predict(df.drop(columns=non_feature_columns))
outlierness = pd.DataFrame({"outlierness": clf.negative_outlier_factor_, "outliers_pred": outliers_pred})

In [ ]:
# Representar el outlierness de cada instancia con un scatter
def plot_raius_outlierness(X: pd.DataFrame, Y:pd.DataFrame, outlierness: pd.DataFrame):
    # Los labels reales se utiilzan únicamente para visualizar
    # En un entorno real no se contaría con Y

    label2color = {
        0: (0, 0, 0),
        1: (162, 78, 255)
    }

    # Para calcular el scree graph
    all_pca = PCA(n_components=len(X.columns))
    all_pca.fit(X)
    # print(f"PLOTTING OUTLIERNESS\nPCA variabilidad: {all_pca.explained_variance_ratio_}\nAutovalores: {all_pca.singular_values_}")
    
    # Reducir a las 2 dimensiones más representativas con PCA
    pca = PCA(n_components=2)
    X_reduced = pca.fit_transform(X)
    radius = (outlierness["outlierness"].max() - outlierness["outlierness"]) / (outlierness["outlierness"].max() - outlierness["outlierness"].min())
    color = Y.map(lambda label: label2color[label])

    # Plotear el scree graph y las instancias-outlierness
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
    ax1.set_title('Local Outlier Factor (LOF)')
    ax1.scatter(X_reduced[:, 0], X_reduced[:, 1], color=color, s=7.0, label="Data points")
    ax1.scatter(X_reduced[:, 0], X_reduced[:, 1], s=1000 * radius, edgecolors="r", facecolors="none", label="Outlier scores")
    ax1.set(xlabel='PCA1', ylabel='PCA2')
    ax1.legend()
    
    ax2.set_title('PCA Scree Graph')
    ax2.plot(range(1,len(all_pca.singular_values_ )+1), all_pca.singular_values_, marker='o', c='blue', label='Variabilidad PCA')
    ax2.set(xlabel='component number', ylabel='Eigen value')
    
    plt.show()

#plot_raius_outlierness(df.drop(columns=non_feature_columns), df.drop(columns='is_metal'), outlierness)